In [68]:
with open('day19_inputs.txt') as f:
    content = [l.strip('\n') for l in f]

for l in content:
    print(l)

0: 1 2
1: "a"
2: 1 3 | 3 1
3: "b"

aba
abb
aab


### Part 1 

Input is in 2 parts, rules (top) and messages to validate (bottom). Task is to return the number of messages that pass rule 0.

Rules apply in one of 3 ways:
1. Apply rules in designated order: e.g. 0: 4 1 5 means to pass rule 0, rules 4, 1, **then** 5 must be passed in order
2. Apple rules in _either_ designated order: e.g. 1: 2 3 | 3 2 means to pass rule 1, rules 2 then 3 **or** 3 then 2 must be passed
3. Match a letter: e.g. 5: "b" means that to pass rule 5, the message must contain b

Given this set of example rules
- 0: 1 2
- 1: "a"
- 2: 1 3 | 3 1
- 3: "b"

Rule 0 is passed by applying rule 1 _then_ rule 2, which means the message must start with "a" then pass rule 2. Rule 2 states that 1 then 3 or 3 then 1 is a valid pass - "ab" or "ba".

In all, this means that rule 1 then rule 2 applied in order means that "aab" or "aba" are valid.

In [69]:
import re

rules = []
messages = []
for l in content:
    print(l)
    rules.append(l) if re.match('[0-9]',l) is not None else messages.append(l)

0: 1 2
1: "a"
2: 1 3 | 3 1
3: "b"

aba
abb
aab


In [79]:
import re
# First step is to parse rules

dict_rules = {}

for r in rules:
    key, sub_rules = r.replace('"','').split(': ')
    if sub_rules.count('|') > 0:
        sub_rules = sub_rules.split(' | ')
        sub_rules = [s.split(' ') for s in sub_rules]
    else:
        sub_rules = sub_rules.split(' ')
    
    dict_rules[int(key)] = sub_rules

for key in dict_rules:
    print(key,':',dict_rules[key])

0 : ['1', '2']
1 : ['a']
2 : [['1', '3'], ['3', '1']]
3 : ['b']


- If dict result is single letter, append to result
- If dict result is list on single keys, loop through and append
- If dict result is list of lists, duplicate results then loop through those keys

a
a(13, 31)
a(ab) a(ba)

In [95]:

def dict_search(keys, dict_to_search, new_sub_rules=None):
    print(f'start recursion, key(s) {keys}')
    
    if new_sub_rules is None:
        new_sub_rules = []
    
    # Case 1, single key to search
    if len(keys) == 1:
        key = keys[0]
        if is_letter(key):
            new_sub_rules.append(key)
        # If key is number, search for results
        else:
            results = dict_to_search[int(key)]
            # If result of this search is a single letter, append to results
            if len(results) == 1 and is_letter(results[0]):
                new_sub_rules.append(results[0])
            # Otherwise, start process once more
            else:
                for r in results:
                    dict_search(r, dict_to_search)
        
    # Case 2, multiple keys
    else:
        if not isinstance(keys[0], list):
            print('non list', keys[0])
            for key in keys:
                dict_search(key, dict_to_search)
        # More complex for lists
        else:
            pass
#             new_sub_rules = [new_sub_rules]*2
#             key1, key2 = key
#             for key in key1:
#                 dict_search(key, dict_to_search)
#             for key in key2:
#                 dict_search(key, dict_to_search)
#             result2.extend(replicate_recur(times - 1, data))
        
    
    print(f'end recursion: new sub rules {new_sub_rules}')
    return new_sub_rules

dict_search([0], dict_rules)

start recursion, key(s) [0]
start recursion, key(s) 1
end recursion: new sub rules ['a']
start recursion, key(s) 2
start recursion, key(s) ['1', '3']
non list 1
start recursion, key(s) 1
end recursion: new sub rules ['a']
start recursion, key(s) 3
end recursion: new sub rules ['b']
end recursion: new sub rules []
start recursion, key(s) ['3', '1']
non list 3
start recursion, key(s) 3
end recursion: new sub rules ['b']
start recursion, key(s) 1
end recursion: new sub rules ['a']
end recursion: new sub rules []
end recursion: new sub rules []
end recursion: new sub rules []


[]

In [90]:
l1 = [0,1]
l1.extend(l1)
l1

[0, 1, 0, 1]

In [58]:
def is_letter(string):
    try:
        return re.match('[a-z]',string) is not None
    except:
        return False
    
new_sub_rules = ['']
def dict_search(keys, dict_to_search):
    print(f'start recursion, key(s) {keys}')
    
    # For cases where we have multiple keys to search, recurse
    if len(keys) > 1:
        for key in keys:
            dict_search(key, dict_to_search)
    else:
        key = keys[0]
        if is_letter(key):
            for i in range(len(new_sub_rules)):
                new_sub_rules[i] += key
        else:
            rules_result = dict_to_search[int(key)]
            print(rules_result)
    
            # If all results are single letters, join into single string
            if all([is_letter(r) for r in rules_result]):
                for i in range(len(new_sub_rules)):
                    new_sub_rules[i] += ''.join(rules_result)
            else:
                for r in rules_result:
                    if is_letter(r):
                        for i in range(len(new_sub_rules)):
                            new_sub_rules[i] += r
                    elif len(r) > 1:
                        new_sub_rules.extend(new_sub_rules)
                        for sub_r in r:
                            dict_search(sub_r, dict_to_search)
                    else:
                        dict_search(r, dict_to_search)
                
    
    print(f'end recursion: new sub rules {new_sub_rules}')

dict_search([0], dict_rules)

start recursion, key(s) [0]
['1', '2']
start recursion, key(s) 1
['a']
end recursion: new sub rules ['a']
start recursion, key(s) 2
[['1', '3'], ['3', '1']]
start recursion, key(s) 1
['a']
end recursion: new sub rules ['aa', 'aa']
start recursion, key(s) 3
['b']
end recursion: new sub rules ['aab', 'aab']
start recursion, key(s) 3
['b']
end recursion: new sub rules ['aabb', 'aabb', 'aabb', 'aabb']
start recursion, key(s) 1
['a']
end recursion: new sub rules ['aabba', 'aabba', 'aabba', 'aabba']
end recursion: new sub rules ['aabba', 'aabba', 'aabba', 'aabba']
end recursion: new sub rules ['aabba', 'aabba', 'aabba', 'aabba']


In [ ]:
['aab','aba']